In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

# Hyper-parameters
user_clip_num = 2
user_pos_num_clip = 2 # only split users with training positive interactions >= user_clip_num * user_pos_num_clip
max_interact_num = 10
like_threshold = 2

In [2]:
data_string = open('../yahoo-origin/user.txt', 'r', encoding = "ISO-8859-1").read().strip().split('\n')
data_dict = {"user_id":[],"item_id":[],"score":[]}
for da in data_string:
    l_string = da.split(',')
    data_dict["user_id"].append(eval(l_string[0]))
    data_dict["item_id"].append(eval(l_string[1]))
    data_dict["score"].append(eval(l_string[2]))

In [3]:
path='./features'
if not os.path.exists(path):
    os.mkdir(path)
path='./sets'
if not os.path.exists(path):
    os.mkdir(path)

In [4]:
train_data_raw = pd.DataFrame(data_dict)
train_data_raw.shape

(311704, 3)

In [5]:
train_data_pos = train_data_raw[train_data_raw.score>3]
train_data_pos.shape

(125077, 3)

In [6]:
train_data_pos.nunique()

user_id    14382
item_id     1000
score          2
dtype: int64

In [7]:
data_string2 = open('../yahoo-origin/random.txt', 'r', encoding = "ISO-8859-1").read().strip().split('\n')
data_dict2 = {"user_id":[],"item_id":[],"score":[]}
for da in data_string2:
    l_string = da.split(',')
    data_dict2["user_id"].append(eval(l_string[0]))
    data_dict2["item_id"].append(eval(l_string[1]))
    data_dict2["score"].append(eval(l_string[2]))

In [8]:
test_data_raw = pd.DataFrame(data_dict2)
test_data_raw.shape

(54000, 3)

In [9]:
for id in test_data_raw.user_id.unique():
    if id not in train_data_pos.user_id.unique():
        test_data_raw = test_data_raw[test_data_raw.user_id!=id]

In [10]:
test_data_raw.shape

(50590, 3)

In [11]:
test_data_pos = test_data_raw[test_data_raw.score>3]
print(test_data_pos.shape)
print(test_data_pos.user_id.nunique())

(4671, 3)
2383


In [ ]:
from tqdm import tqdm
item_id_map = {}
user_id_map = {}
userID = 0
itemID = 0
big_pos_list = []
train_data = train_data_pos.copy(deep=True)
for i in tqdm(range(train_data.shape[0])):
    if train_data.iloc[i,0] in user_id_map:
        train_data.iloc[i,0] = user_id_map[train_data.iloc[i,0]]
    else:
        uID = userID
        user_id_map[train_data.iloc[i,0]]=uID
        train_data.iloc[i,0] = uID
        userID += 1
    #print(train_data.iloc[i,1])
    if train_data.iloc[i,1] in item_id_map:
        # print("train_data.iloc[i,1]",train_data.iloc[i,1])
        # print("item_id_map:",item_id_map)
        train_data.iloc[i,1] = item_id_map[train_data.iloc[i,1]]
    else:
        iID = itemID
        item_id_map[train_data.iloc[i,1]]=iID
        train_data.iloc[i,1] = iID
        itemID += 1  
# train_data

 99%|███████████████████████████████████████████████████████████████████████▌| 124424/125077 [00:27<00:00, 4515.89it/s]

In [ ]:
train_data_raw

In [ ]:
# added for splitting users: only split training data, keep testing data untouched.
big_matrix = train_data_raw
user_all_dict = {} # to record every exposure interaction with time and watch_ratio(here only with score)
for i in tqdm(range(big_matrix.shape[0])):
    if big_matrix.iloc[i,0] in user_id_map:
        uID = user_id_map[big_matrix.iloc[i,0]]
        
        # this is a recorded positive interaction
        if big_matrix.iloc[i,2]>=4: 
            iID = item_id_map[big_matrix.iloc[i,1]]
#             assert [uID, iID] in big_pos_list # double check this is recorded
            
            if uID not in user_all_dict:
                user_all_dict[uID] = {'exposure':[], 'like':[]}
            user_all_dict[uID]['exposure'].append([iID, big_matrix.iloc[i,2]])
            user_all_dict[uID]['like'].append([iID, big_matrix.iloc[i,2]])
            
        # this is a negative interaction
        else:
            if big_matrix.iloc[i,1] in item_id_map: 
                iID = item_id_map[big_matrix.iloc[i,1]]
            # this is a new item unmapped
            else: 
                iID = len(item_id_map)
                item_id_map[big_matrix.iloc[i,1]] = iID
                
            if uID not in user_all_dict:
                user_all_dict[uID] = {'exposure':[], 'like':[]}
            user_all_dict[uID]['exposure'].append([iID, big_matrix.iloc[i,2]])
    else:
        continue # this user is deleted in previous MF data



In [ ]:
embed_item = np.load("./embed_item_MF_yahoo_512factor_num_0.0001lr_256bs_0.0dropout.npy")

import pandas as pd  
from sklearn.cluster import KMeans  
import matplotlib.pyplot as plt  
from tqdm import tqdm
category_num_new = 5
kmeans_model = KMeans(category_num_new, random_state=1)
kmeans_model.fit(embed_item)
labels = kmeans_model.labels_

# create item_feature_dict file CHANGED!!!
item_feature_dict = {}
for item in range(embed_item.shape[0]):
    item_feature_dict[item] = [labels[item]]

np.save('./features/item_feature_dict.npy', np.array(item_feature_dict))

# create category_list file
category_list = list(range(category_num_new))
np.save('./features/category_list.npy', np.array(category_list))

In [ ]:
test_data0 = test_data_pos.copy(deep=True)
for i in tqdm(range(test_data0.shape[0])):
    if test_data0.iloc[i,0] in user_id_map:
        test_data0.iloc[i,0] = user_id_map[test_data0.iloc[i,0]]
    else:
        print('u',uID)
    if test_data0.iloc[i,1] in item_id_map:
        test_data0.iloc[i,1] = item_id_map[test_data0.iloc[i,1]]
    else:
        print('i',iID)
# test_data0

In [ ]:
train_dict=train_data.sort_values('user_id').groupby('user_id')
train_user_id=train_data.user_id.unique()
training_dict=dict()
for i in train_user_id:
    training_dict[i]=train_dict.get_group(i).item_id.values.tolist()
training_list = []
for u in training_dict:
    for i in training_dict[u]:
        training_list.append([u,i])

In [ ]:
# split users and generate new big_pos_dict and big_pos_list.
big_pos_dict = dict(training_dict)
big_pos_list = list(training_list)
# a function to calc T given exposure items and item_category_dict&category_list
def calc_T(t_seq, item_category, category_list):
    cate_dis = [0] * (max(category_list)+1)
    for item in t_seq:
        for c in item_category[item]:
            cate_dis[c] += 1/len(item_category[item])
    cate_dis = [c/len(t_seq) for c in cate_dis]
    return cate_dis

# a function to calc Y given exposure items, liked items, and item_category_dict&category_list
def calc_Y(t_seq, next_d, item_category, category_list):
    exposure_dis = [0]* (max(category_list)+1)
    like_dis = [0]* (max(category_list)+1)
    for item in t_seq:
        for c in item_category[item]:
            exposure_dis[c] += 1/len(item_category[item])
    for item in next_d:
        for c in item_category[item]:
            like_dis[c] += 1/len(item_category[item])
    ctr_dis = [like_dis[c]/exposure_dis[c] if exposure_dis[c] > 0 else 0 for c in range(len(exposure_dis))]
    return ctr_dis
    
subuser_all_dict = {} # subuserID
user_subuser_map = {}
user_subuser_exp_dict = {} # save exposure data of users and subusers
user_num = len(user_id_map)
assert user_num not in big_pos_dict

for uID in tqdm(user_all_dict):
    pos_num = len(user_all_dict[uID]['like'])
    
    # save exposure data of uID
    user_subuser_exp_dict[uID] = [entry[0] for entry in user_all_dict[uID]['exposure']]
    
    # only split users with postive len > user_clip_num * user_pos_num_clip
    if pos_num < user_clip_num * user_pos_num_clip:
        continue
    pos_num_clip = pos_num//user_clip_num
    for i in range(user_clip_num-1):
        
        subuser_id = user_num
        user_num += 1
        user_subuser_map[subuser_id] = uID
        subuser_all_dict[subuser_id] = {}
        
        # find like history in current clip
        d = user_all_dict[uID]['like'][:(i+1)*pos_num_clip]
        
        # find treatment in next clip
        st = user_all_dict[uID]['exposure'].index(user_all_dict[uID]['like'][(i+1)*pos_num_clip])
        if i != user_clip_num-2: # not last clip
            ed = user_all_dict[uID]['exposure'].index(user_all_dict[uID]['like'][(i+2)*pos_num_clip])
            t_seq = user_all_dict[uID]['exposure'][st:ed]
        else:
            t_seq = user_all_dict[uID]['exposure'][st:]
            
        # find like history in next clip
        if i != user_clip_num-2: # not last clip
            next_d = user_all_dict[uID]['like'][(i+1)*pos_num_clip:(i+2)*pos_num_clip]
        else:
            next_d = user_all_dict[uID]['like'][(i+1)*pos_num_clip:]
        
        # save exposure data of subuser_id: find the last liked item and save its exposure.
        exposure_index = user_all_dict[uID]['exposure'].index(d[-1])
        user_subuser_exp_dict[subuser_id] = user_all_dict[uID]['exposure'][:exposure_index+1]
        
        d = [entry[0] for entry in d]
        next_d = [entry[0] for entry in next_d]
        t_seq = [entry[0] for entry in t_seq]
        
        T = calc_T(t_seq, item_feature_dict, category_list)
        Y = calc_Y(t_seq, next_d, item_feature_dict, category_list)

        subuser_all_dict[subuser_id]['D'] = d
        subuser_all_dict[subuser_id]['T'] = T
        subuser_all_dict[subuser_id]['Y'] = Y
                
        # update big_pos_dict and big_pos_list
        for item in d:
            if subuser_id not in big_pos_dict:
                big_pos_dict[subuser_id] = []
            big_pos_dict[subuser_id].append(item)
            big_pos_list.append([subuser_id, item])

        if uID == 0:
            print(subuser_all_dict[subuser_id])

In [ ]:
np.save('./sets/training_dict.npy',np.array(big_pos_dict)) # 保存为.npy格式
np.save('./sets/training_list.npy',np.array(big_pos_list)) # 保存为.npy格式
np.save('./features/user_subuser_exp_dict.npy', np.array(user_subuser_exp_dict))
np.save('./features/user_subuser_map.npy', np.array(user_subuser_map))
np.save('./features/subuser_info.npy', np.array(subuser_all_dict))

In [ ]:
import math
import numpy as np
def split_pandas_data_with_ratios(data, ratios, shuffle=False):
    """
    按输入的比例列表对数据进行切分
    Args:
        data (pd.DataFrame): Pandas DataFrame 格式的数据.
        ratios (list of floats): 切分比例的列表，和需要是 1.
        shuffle (bool): 是否需要洗牌.

    Returns:
        list: 切分后的数据列表，列表中的元素类型为 pd.DataFrame .
    """
    # 检查 切分比例的列表是否和为 1
    if math.fsum(ratios) != 1.0:
        raise ValueError("切分比例需要其和为1")
        
    # 累加求和，例如 [0.7,0.15,0.15] 累计求和为 [0.7, 0.85, 1]
    # 然后剔除最后一个值，即结果为 [0.7, 0.85]
    split_index = np.cumsum(ratios).tolist()[:-1]
    
    # 是否洗牌
    if shuffle:
        # 不放回的从原始数据中随机取数据，直到到达frac设置的比例
        data = data.sample(frac=1)
    # 切分
    #print("data_size:",len(data))

    splits = np.split(data, [round(x * len(data)) for x in split_index])

    #对于valid为0的处理（只能处理训练集和验证集
    # if(len(splits[0])==1):
    #     splits[1]=splits[0].copy()

    split_index_small=[ 0.5, 1 ]
    if(len(splits[1])==0):
        splits = np.split(data, [round(x * len(data)) for x in split_index_small])

    for i in range(len(ratios)):
        splits[i]["split_index"] = i
    return splits

In [ ]:
df_grouped = test_data0.groupby('user_id')
splits = []
ratio=[0.5,0.5]

# 对于每一名用户
for name, group in df_grouped:
    if(df_grouped.get_group(name).shape[0]<2):
        continue
    group_splits = split_pandas_data_with_ratios(df_grouped.get_group(name), ratio)
    # 把切分好的数据再合并，数据值只多了 split_index 这一列
    concat_group_splits = pd.concat(group_splits)
    splits.append(concat_group_splits)

# 把所有用户的数据合并
splits_all = pd.concat(splits)

# 按照 split_index 标记把数据切分出来
splits_list = [
    splits_all[splits_all["split_index"] == x].drop("split_index", axis=1)
    for x in range(len(ratio))
]
valid_data=splits_list[0]
test_data=splits_list[1]

In [ ]:
train_user_id=train_data.user_id.unique()
validation_id=valid_data.user_id.unique()
test_id=test_data.user_id.unique()
print(list(validation_id)==list(test_id))
for i in validation_id:
    if i not in train_user_id:
        print(i," not in training set")

In [ ]:
valid_dict=valid_data.groupby('user_id')
valid_user_id=valid_data.sort_values('user_id').user_id.unique()
validation_dict=dict()
for i in valid_user_id:
    validation_dict[i]=valid_dict.get_group(i).item_id.values.tolist()
np.save('./sets/validation_dict.npy',np.array(validation_dict)) # 保存为.npy格式

In [ ]:
test_dict=test_data.groupby('user_id')
test_user_id=test_data.sort_values('user_id').user_id.unique()
testing_dict=dict()
for i in test_user_id:
    testing_dict[i]=test_dict.get_group(i).item_id.values.tolist()
np.save('./sets/testing_dict.npy',np.array(testing_dict)) # 保存为.npy格式

In [ ]:
pop_rank = dict(pd.DataFrame(training_list)[1].value_counts())
print(len(pop_rank))
for item in item_id_map:
    i = item_id_map[item]
    if i not in pop_rank:
        pop_rank[i] = 0
print(len(pop_rank))

item_rank_dict = {} #id:rank
rank = 0
for element in pop_rank:
    item_rank_dict[element] = rank
    rank += 1
np.save('./features/item_rank_dict.npy', np.array(item_rank_dict))

In [ ]:
category_num = len(category_list)
topk_category = 3
user_fml_cat_big = {}
for user in training_dict:
    category_cnt = [0] * category_num
    for item in training_dict[user]:
        for cat in item_feature_dict[item]:
            category_cnt[cat] += 1
    sorted_list = list(np.argsort(category_cnt))
    fml_cat = sorted_list[-topk_category:]
    user_fml_cat_big[user] = fml_cat
np.save('./features/user_fml_cat.npy', np.array(user_fml_cat_big))

In [ ]:
user_feature_file = {}
for userID in big_pos_dict:
    user_feature_file[userID] = [['U'+str(userID)]]
    user_feature_file[userID].append([str(1)]*len(user_feature_file[userID][0]))

item_feature_file = {}
for itemID in item_feature_dict:
    item_feature_file[itemID] = [['I'+str(itemID)]]
    item_feature_file[itemID].append(['1'])
    for cate in range(0, len(category_list)):
        if cate in item_feature_dict[itemID]:
            item_feature_file[itemID][0].append('IC'+str(cate))
            item_feature_file[itemID][1].append(str(round(1.0/len(item_feature_dict[itemID]), 2)))
        else:
            item_feature_file[itemID][0].append('IC'+str(cate))
            item_feature_file[itemID][1].append(str(0))
np.save('./features/user_feature_file.npy', np.array(user_feature_file))
np.save('./features/item_feature_file.npy', np.array(item_feature_file))

In [ ]:
item_cat_vec = {}
for itemID in item_feature_dict: 
    item_cat_vec[itemID] = [0] * category_num
    cat_num = len(item_feature_dict[itemID])
    for cat in item_feature_dict[itemID]:
        item_cat_vec[itemID][cat] = 1 / np.sqrt(cat_num)
np.save('./features/item_cat_vec.npy', np.array(item_cat_vec))